# SLM calibration

This file is used to generate , display and measure grayscale images for calibrating the phase shift of a phase SLM.

The SLM has a resolution of 800x600 px, each 32x32um^2 with a fill factor of 85%

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ximea import xiapi
import cv2
import threading
from screeninfo import get_monitors
import time

## Image acquisition example

In [ ]:
from ximea import xiapi
import matplotlib.pyplot as plt
import numpy as np


#CIMAU2430046
cam = xiapi.Camera(dev_id=0)
print("Opening camera...")
cam.open_device_by_SN('CIMAU2430046')
width = cam.get_width()
height = cam.get_height()
print("Camera open (if it worked)")

sensor_bit_depth = cam.set_sensor_bit_depth('XI_BPP_12')
sensor_bit_depth = cam.get_sensor_bit_depth()
print('Sensor bit depth: ', sensor_bit_depth)
cam.set_gain(0) 
cam.set_exposure(20000)
img = xiapi.Image()
cam.set_trigger_source('XI_TRG_SOFTWARE') 
print('Starting data acquisition...')
cam.start_acquisition()
cam.set_trigger_software(1) 
cam.get_image(img)
print('blah')

data_raw = img.get_image_data_raw()
data = list(data_raw)

cam.stop_acquisition()
cam.close_device()

npdata = np.reshape(data,(height, width))

plt.imshow(npdata,cmap ='grey')
plt.show()

## Display example in a thread with updates possible called by the "main thread"

In [ ]:
import cv2
import threading
from screeninfo import get_monitors
import time

from calibration import FullscreenDisplay

display = FullscreenDisplay(path1+str(index)+".bmp", monitor_index=1)
display.show()
RUNNING = True
print("Program running... window stays open until we close it.")
while RUNNING == True:
    user_input = input()
    if user_input=='e':
        display.update_image("temp2.jpg")
        time.sleep(2)
        print("Closing display")
        display.close()
        RUNNING = False
# Now we send a command to close the window
# print("Closing display now...")
# display.close()
print("Program finished.")


## Combining the two of them

In [ ]:
from calibration import generate_frames, FullscreenDisplay, Acquisition
import time
import keyboard

SN = 'CIMAU2430046'
path1 = "data/calibration_"
path2 = "data/measurements/mes_"
ext = ".bmp"
monitor_index = 1 # 0:main, 1:secondary, 3:tertiary, ...

exposure = 35000            # us
exposure_s = exposure*1e-6
Stabilisation_time = 50e-3  # ms
N = 255
HYSTERESIS = True
hysteresis_index_offset = 1000

index = 0


generate_frames(N, path1)

RUNNING = True

print(path1+str(index)+".bmp")
display = FullscreenDisplay(path1+str(index)+".bmp", monitor_index)
# display.show()
display.show_mp()
time.sleep(Stabilisation_time)

print("Starting acquisition")
acquisition = Acquisition(exposure, path2)
print("Acquisition ready")
tic = time.time()
tac = tic
print("Press \'e\' for emergency exit")
while RUNNING:
    if tac-tic >= Stabilisation_time: 
        acquisition.get(index)  # makes the programm wait on it
        if index == N - 1:
            RUNNING = False
        else :
            display.update_image(path1+str(index)+ext)
            display.update_image_mp(path1+str(index)+ext)
            tic = tac
            index += 1
      
    tac = time.time()
    if keyboard.is_pressed('e'):
        print('Process abruptly terminated. Restart kernel.')
        RUNNING = False
        HYSTERESIS = False
    time.sleep(0.001)
    
tic = time.time()
tac = tic
while HYSTERESIS:
    if tac-tic >= Stabilisation_time:
        acquisition.get(index+hysteresis_index_offset)# makes the programm wait on it
        if index == 0:
            HYSTERESIS = False
        else :
            display.update_image_mp(path1+str(index)+ext)
            tic = tac
            index -= 1
    

    tac = time.time()
    if keyboard.is_pressed('e'):
        print('Process abruptly terminated. Restart kernel.')
        RUNNING = False
    time.sleep(0.001)

print("Terminating processes")
acquisition.terminate()
display.close()
